# Week 04 Assignment weather data

Welcome to week four of this course programming 1. You will to organise your data into the required format and apply smoothing. In this assignment we will work with weatherdata from the KNMI. A subset of weatherdata is for you available in the file: `KNMI_20181231.csv`. The data consist of several stations with daily weather data of several years. Your task is to make a plot similar to the plot below.


<img src="images/weather.png" alt="drawing" width="400"/>


Furthermore the plot needs the following enhancements

1. proper titles and ticks
2. a slider widget selecting a particular year or all years
3. lines need to be smoothed
3. legends needs to be added

Use your creativity. Consider colors, alpha settings, sizes etc. 

Learning outcomes

- load, inspect and clean a dataset 
- reformat dataframes
- apply smoothing technologies
- visualize timeseries data

The assignment consists of 6 parts:

- [part 1: load the data](#0)
- [part 2: clean the data](#1)
- [part 3: reformat data](#2)
- [part 4: smooth the data](#3)
- [part 5: visualize the data](#4)
- [part 6: Challenge](#5)

Part 1 and 5 are mandatory, part 6 is optional (bonus)
To pass the assingnment you need to a score of 60%. 


---

<a name='0'></a>
## Part 1: Load the data

Either load the dataset `KNMI_20181231.csv` or `KNMI_20181231.txt.tsv`. The dataheaders contain spaces and are not very self explainable. Change this into more readable ones. Select data from station 270. Select only the mean, minimum and maximum temperature. The data should look something like this:


In [106]:
import numpy as np
import pandas as pd

# Read file and select relevant columns
df = pd.read_csv('KNMI_20181231.txt.tsv', sep="\t", header=63)
df[["station", "date", "tmean", "tmin", "tmax", "pop"]] = df["# STN,YYYYMMDD,   TG,   TN,   TX,   SQ,   DR,   RH"].str.split(',', n=5, expand=True)
df = pd.DataFrame.drop(df, columns=["# STN,YYYYMMDD,   TG,   TN,   TX,   SQ,   DR,   RH", "pop"])
df = df[df["station"].apply(pd.to_numeric, errors='coerce') == 270]
df.head()

,station,date,tmean,tmin,tmax
97642,270,20000101,42,-4,79
97643,270,20000102,55,33,74
97644,270,20000103,74,49,89
97645,270,20000104,46,22,75
97646,270,20000105,41,14,56


---

<a name='1'></a>
## Part 2: Clean the data

The data ia not clean. There are empty cells in the dataframe which needs to be replaced with NaN's and the temperature is in centidegrees which needs to be transformed into degrees. The date field needs a datetime format. For visualization convience we would like to remove the leap year. Conduct the cleaning.

In [107]:
# Replace cells with spaces to NaN
df.columns = df.columns.str.replace(' ', '')

# Change data formats
df[["tmean", "tmin", "tmax"]] = df[["tmean", "tmin", "tmax"]].apply(pd.to_numeric, errors='coerce', axis=1)
df["date"] = pd.to_datetime(df["date"])

# Change temperatures to celcius degrees
df[["tmean", "tmin", "tmax"]] = df[["tmean", "tmin", "tmax"]].div(10)
print("Length of df including leap years: {:.3f}".format(len(df)))

# Remove leap year
df = df[df["date"].dt.is_leap_year == False]
print("Length of df excluding leap years: {:.3f}".format(len(df)))

# Sort data and select change datatypes
df.sort_values("date")
df.reset_index(drop=True)
df.head()

Length of df including leap years: 6940.000
Length of df excluding leap years: 5110.000


,station,date,tmean,tmin,tmax
98008,270,2001-01-01,2.1,0.4,3.8
98009,270,2001-01-02,6.3,3.7,8.5
98010,270,2001-01-03,5.3,2.6,7.9
98011,270,2001-01-04,6.5,4.6,7.9
98012,270,2001-01-05,6.6,5.6,8.3


<details>    
<summary>
    <font size="3" color="darkgreen"><b>Hints</b></font>
</summary>
<ul><li>pd.to_datetime(df['Date'].astype(str), format='%Y%m%d')</li>
    <li>regex for empty cells = `^\s*$` </li>
    <li>remove month == 2 & day == 29</li> 
</ul>
</details>

In [108]:
#Test your outcome
df.info()

## Why does station need to be an integer if I do no further actions with it?

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5110 entries, 98008 to 104581
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   station  5110 non-null   object        
 1   date     5110 non-null   datetime64[ns]
 2   tmean    5110 non-null   float64       
 3   tmin     5110 non-null   float64       
 4   tmax     5110 non-null   float64       
dtypes: datetime64[ns](1), float64(3), object(1)
memory usage: 239.5+ KB


### Expected outcome

---

<a name='2'></a>
## Part 3: Reform your data

First we will split the data in data from 2018 and data before 2018. Best is to split this in two dataframes. 
Next we need for the non 2018 data the minimum values for each day and the maximum values for each day. So we look for the minimum value out of all january-01 minimum values (regardless the year). Create a dataframe with 365 days containing the ultimate minimum and the ultimate maximum per day. 


In [200]:
df_2018 = df[df['date'].dt.year == 2018]
df_pre2018 = df[df['date'].dt.year < 2018]

def month_day(df_multipleyears):
    # Create df of relevant columns and delete year data
    ext_vals= df_multipleyears[["date", "tmin", "tmax"]]
    ext_vals['day'] = ext_vals['date'].dt.day
    ext_vals['month'] = ext_vals['date'].dt.month
    ext_vals = pd.DataFrame.drop(ext_vals, columns=["date"])

    # Create 2 df's: one with max and other min values per day
    tmaxx = ext_vals.groupby(["month", "day"]).max()
    tminn = ext_vals.groupby(["month", "day"]).min()

    # Merge the two df's and rename accordingly
    max_min = pd.merge(tmaxx, tminn, on=["month", "day"], how="outer")
    max_min = pd.DataFrame.drop(max_min, columns=["tmin_x", "tmax_y"], axis=1)
    max_min = max_min.rename(columns={"tmax_x": "tmax", "tmin_y": "tmin"})
    print(max_min)
    return max_min

In [209]:
#Test your code
def test_reformed(df):
    #
    df = df[(df.date.dt.year > 2007) & (df.date.dt.year < 2018)]
    ref_df = month_day(df)
    return ref_df

test = test_reformed(df)

           tmax  tmin
month day            
1     1     8.6  -5.8
      2    10.2  -7.5
      3    10.7 -12.6
      4     9.2  -4.1
      5     8.4  -6.0
...         ...   ...
12    27   11.7  -2.0
      28   10.9  -4.2
      29    8.6  -5.0
      30    9.5  -6.7
      31   11.8   0.2

[365 rows x 2 columns]


C:\Users\jarom\AppData\Local\Temp/ipykernel_10072/397922807.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ext_vals['day'] = ext_vals['date'].dt.day


<details>    
<summary>
    <font size="3" color="darkgreen"><b>Hints</b></font>
</summary>
<ul><li>use the dt.month and dt.day to groupby</li>
</ul>
</details>

### Expected outcome
Note, the layout or names my differ, but the length should be 365 and the minimum values should be the same

---

<a name='3'></a>
## Part 4: Smooth the data

Make a function that takes an array or a dataframe column and returns an array of smoothed data. Explain in words why you choose a certain smoothing algoritm


In [213]:
#your code here
def smooth(df):
    df = df.rolling(5).mean()
    return df

smoothed = smooth(test)

**your motivation here** I used a simple moving average to smooth the data, because this method smooths out short-term fluctuations and highlights longer-term trends. This can be especially valuable to display the seasonal changes in temperature data we have.

---

<a name='4'></a>
## Part 5: Visualize the data

Plot the mean temperature of the year 2018. Create a shaded band with the ultimate minimum values and the ultimate maximum values from the multi-year dataset. Add labels, titles and legends. Use proper ranges. Be creative to make the plot attractive. 



<details>    
<summary>
    <font size="3" color="darkgreen"><b>Hints</b></font>
</summary>
<ul><li>use from bokeh.models import Band</li>
    <li>use ColumnDataSource to parse data arrays</li>
    <li>look for xaxis tick formatters</li>
</ul>
</details>

In [225]:
from bokeh.io import output_notebook
from bokeh.plotting import figure, show
from bokeh.models import Band
from bokeh.layouts import gridplot
from bokeh.plotting import ColumnDataSource
import numpy as np
output_notebook()

# function to make a plot for temperature data
def make_plot(df):
    # Smooth data of df
    df2 = smooth(df)
    df2 = df2.fillna(method='bfill')
    
    df = pd.concat([df, df2], axis=1)
    source = ColumnDataSource(df.reset_index())

    TOOLS = "pan,wheel_zoom,box_zoom,reset,save"
    p = figure(tools=TOOLS, x_axis_type='datetime')
    
    p.scatter(x='date', y='tmean', line_color=None, fill_alpha=0.3, size=5, source=source, color="green")
    
    band = Band(base="date", lower='tmin', upper='tmax', source=source, level='underlay',
            fill_alpha=1.0, line_width=2, line_color='maroon')
    p.add_layout(band)

    p.title.text = "Temperature of 2018 in Leeuwarden"
    p.xgrid[0].grid_line_color=None
    p.ygrid[0].grid_line_alpha=0.5
    p.xaxis.axis_label = "Date"
    p.yaxis.axis_label = "Temperature in Centidegrees"
    p.grid.grid_line_color="white"

    show(p)
    return p

make_plot(df_2018)


Loading BokehJS ...

Figure(id='3068', ...)

---

<a name='5'></a>
## Part 6: Challenge

Make a widget in which you can select the year range for the multiyear set. Add this to your layout to make the plot interactive. Add another widget to select or deselect the smoother. Inspiration: https://demo.bokeh.org/weather